In [1]:
import pandas as pd
import torch
from tqdm import tqdm
torch.cuda.empty_cache()
import numpy as np
from sklearn.metrics import f1_score

from torch.utils.data import TensorDataset
from transformers import BertTokenizer

from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import time


In [2]:
labels_dict = {'ANGRY':0, 
          'DISGUST':1, 
          'FEAR':2, 
          'HAPPY':3, 
          'SAD':4, 
          'SURPRISE':5}

# Enter your sequence here:
sequence = [
    "I'M SO GONNA KILL YOU YOU PRICK!!!",
    "Yucks.. gross...",
    "I'm hiding at home till this is over.",
    'I LOVE DONUTS',
    "A pity... a pity.",
    "WHAT?! When did this happen!!!"    
]

In [3]:
# Create tokenizer object
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

In [4]:
# Create model and load weights
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels = 6,
                                                      output_attentions=False,
                                                      output_hidden_states=False)
PATH = './bert emotion/BERT_emotion_epoch_10.pt'
model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')), strict=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [6]:
def evaluate(valid_dataloader):
    model.eval()

    total_eval_loss = 0
    y_hat, y = [], []

    for batch in tqdm(valid_dataloader):
        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids' : batch[0],
                'attention_mask': batch[1],
                'labels' : batch[2]
                }
        
        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        y_hat.append(logits)
        y.append(label_ids)

    avg_eval_loss = total_eval_loss/len(valid_dataloader) 

    y_hat = np.concatenate(y_hat, axis=0)
    y = np.concatenate(y, axis=0)
            
    return avg_eval_loss, y_hat, y

In [7]:
valid_encode = tokenizer.batch_encode_plus(
    sequence,
    padding=True,
    truncation=True,
    add_special_tokens=True,
    max_length=256,   
    return_tensors='pt'
)
valid_input = valid_encode['input_ids']
valid_attention = valid_encode['attention_mask']
# The line below is a hack. I'm not entirely sure how to work
# with PyTorch to predict using purely a sequence.
labels = [0]*len(valid_input)
valid_labels = torch.tensor(labels)

valid_data = TensorDataset(valid_input,
                          valid_attention,
                        valid_labels)

valid_dataloader = DataLoader(valid_data,
                              sampler = SequentialSampler(valid_data),
                              batch_size = 8)


In [8]:
# Actual prediction happens here
_, predictions, actual = evaluate(valid_dataloader)


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.80it/s]


In [10]:
# Printing of labels sequence 
labs = {v:k for k,v in labels_dict.items()}
for i in tqdm(range(len(predictions))):
    print('Label: {}, Sequence:{}'.format(labs[np.argmax(predictions[i])], sequence[i]))
    print()

100%|██████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 6003.30it/s]

Label: ANGRY, Sequence:I'M SO GONNA KILL YOU YOU PRICK!!!

Label: DISGUST, Sequence:Yucks.. gross...

Label: FEAR, Sequence:I'm hiding at home till this is over.

Label: HAPPY, Sequence:I LOVE DONUTS

Label: SAD, Sequence:A pity... a pity.

Label: SURPRISE, Sequence:WHAT?! When did this happen!!!

